In [65]:
#Mutation based fuzzer


import random
import string

#config
MAX_NUMBER_OF_MUTATORS = 10
INSERT_MAX_LENGTH = 10
DELETE_MAX_LENGTH = 10

ALL_ASCII = []
for i in range(0, 128):
    ALL_ASCII.append(chr(i))

def mutator_insert(input_data):
    length = random.randrange(0, INSERT_MAX_LENGTH)
    position = random.randrange(0, len(input_data))
    #random_bytes = bytearray()
    random_bytes = ""
    for i in range(0, length):
        #random_bytes += bytes([random.randrange(0, 255)])
        random_bytes += random.choice(string.printable)
    return input_data[:position] + random_bytes + input_data[position:]

def mutator_delete(input_data):
    length = len(input_data)
    position = random.randrange(8, length)
    count = random.randrange(0, min(length, DELETE_MAX_LENGTH))
    return input_data[:position] + input_data[position+count:]
        
#active_mutators=[mutator_insert, mutator_delete]
active_mutators=[mutator_insert]

def mutate_corpus(corpus,run_id):
    random.seed(run_id)
    active_corpus=random.choice(corpus)
    return mutate(active_corpus, run_id)

def mutate(input_data, run_id):
    random.seed(run_id)
    number_of_mutations = random.randint(0, MAX_NUMBER_OF_MUTATORS)
    for i in range(0, number_of_mutations):
        mutator = random.choice(active_mutators)
        input_data = mutator(input_data)
    return input_data

    

In [66]:
#mutate(bytes(10), 1)
#import glob
#print(glob.glob("./corpus/*"))
#corpus_files = glob.glob("./corpus/*")
#corpus=[]

#for file in corpus_files:
#    with open(file, "rb") as f:
#        corpus.append(f.read())

        
#print(mutate_corpus(corpus,1))
 #####:   16:            if(password[2] == 'c'){
    #####:   17:                if(password[3] == 'd #####:   16:            if(password[2] == 'c'){
    #####:   17:                if(password[3] == 'd'){
    #####:   18:                    if(password[4] == 'e'){
    #####:   19:                        if(password[5] == 'f'){
    #####:   20:                            if(password[6] == 'g'){
    #####:   21:       '){
    #####:   18:                    if(password[4] == 'e'){
    #####:   19:                        if(password[5] == 'f'){
    #####:   20:                            if(password[6] == 'g'){
    #####:   21:       
base_corpus = ["Test", ]

In [67]:
import subprocess

#def runner(testcase):
#    with open("testcase.json", "wb") as f:
#        f.write(testcase)
#        f.close()
#    result = subprocess.run(["./coverage", testcase], shell=False, timeout=1)

def runner(testcase):
    result = subprocess.run(["./coverage", testcase], shell=False, timeout =1)
    return result.returncode

In [68]:
runner(b"asdf")

0

#Fuzzing

In [ ]:
#!clang -fsanitize=address cJSON.c cJSON-fuzzer.c -o cJSON-fuzzer -lm
!rm coverage.gcda 
!rm coverage.gcno
!gcc --coverage coverage.c -o coverage


run_id =0 
current_coverage=0
while True:
    testcase = mutate_corpus(corpus, run_id)
    result=runner(testcase)
    
    r = subprocess.run(["gcov", "coverage"], stdout=subprocess.PIPE)
    #print(r.stdout)
    coverage =int(float(r.stdout.decode('utf-8').split('Lines executed:')[1].split('% of')[0]))
    
    #check if coverage increased
    if(coverage > current_coverage):
        current_coverage = coverage
        base_corpus = [testcase]
        print("Current coverage - " + str(current_coverage) + "%")
    
    if result != 0:
        print(f"Success : {run_id}")
        
    run_id +=1

Current coverage - 33%
Current coverage - 40%


In [ ]:
#testcase = mutate_corpus(corpus, 1931)
#result=runner(testcase)
#print(testcase)
#print(result)